In [2]:
import datetime

class Order:
    def __init__(self, quantity, side, price=None):
        if not isinstance(quantity, int) or quantity <= 0: 
            raise ValueError("Quantity must be a positive integer")
        if not side in ["Buy","Sell"]:
            raise ValueError("side must be either Buy or Sell")
        if price is not None and (not isinstance(price, (int,float)) or price <= 0): 
            raise ValueError("Price must be a positive integer or a float or None")
        
        self.quantity = quantity
        self._side = side
        self.price = price
        self.status="ON"
        print(self.quantity)
        
        if self.price == None:
            self.order_type="Market Order"
        else:
            self.order_type="Limit Order"
            
    @property
    def side(self):
        return self._side
#-----------------------------------------------------------------------------------
class LOB:
    def __init__(self):
        self.orders=[]
    
    def add(self,order):
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")
        self.orders.append({"entry":order,"position":datetime.datetime.now().timestamp()})
        
    def canmatch(self,order,sign=1):
        p=False
        q=False
        price=None
        quantity=None
        position=None
        match_prices = []
        match_quantity = []
        
        
        if len(self.orders)>0:
            if order.side=="Buy":
                listoforders = [old_order for old_order in sorted(self.orders, key=lambda old_order: \
                                (old_order["entry"].price,old_order["position"])) if old_order["entry"].side =="Sell"]
                sign=-1
            elif order.side=="Sell":
                listoforders = [old_order for old_order in sorted(self.orders, key=lambda old_order: \
                                (old_order["entry"].price,old_order["position"])) if old_order["entry"].side =="Buy"]

            if order.order_type=="Limit Order":  
                print("LOB_LO")
                for old_order in listoforders:
                    if order.price*sign <= old_order["entry"].price*sign:
                        p=True
                        match_prices.append(min(order.price,old_order["entry"].price))
                        if order.quantity <= old_order["entry"].quantity:
                            q=True
                            match_quantity.append((min(order.quantity,old_order["entry"].quantity),old_order["position"]))
                        elif order.quantity > old_order["entry"].quantity:
                            match_quantity.append((min(order.quantity,old_order["entry"].quantity),old_order["position"]))
                        if q==True:
                            position=old_order["position"]
                            break
                        else:
                            continue
            elif order.order_type=="Market Order":
                print("LOB_MO")
                for old_order in listoforders:
                        match_prices.append(max(0,old_order["entry"].price))
                        p=True
                        if order.quantity <= old_order["entry"].quantity:
                            q=True
                            match_quantity.append((min(order.quantity,old_order["entry"].quantity),old_order["position"]))
                        elif order.quantity > old_order["entry"].quantity:
                            match_quantity.append((min(order.quantity,old_order["entry"].quantity),old_order["position"]))
                            
                        if q==True:
                            position=old_order["position"]
                            break
                        else:
                            continue

            if p == True:
                price=min([prix for prix in sorted(match_prices)])
                quantity,position=[qty for qty in sorted(match_quantity)][-1]
                
        elif len(self.orders)==0:
            p,q,price,quantity,position=(False,False,None,None,None)
            
        return p,q,price,quantity,position
        
    def check(self, order):
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")

        p,q=self.canmatch(order)[:2]

        if p==True and q==True:
            decision = True
        elif p==True and q==False:
            decision = "Partial"
        else:
            decision = False  
            
        return decision
    
    def match(self,order):
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")
        if self.check(order) == True or self.check(order) == 'Partial':
            return tuple(self.canmatch(order)[2:4])
    
    def adjust_book(self,order):
        trade=False
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")
            
        if self.check(order)==True or self.check(order)=='Partial':
            price,quantity,position = tuple(self.canmatch(order)[2:])
            res_quantity=order.quantity-quantity
            perc=order.quantity/quantity
        
            if res_quantity>0 and perc>1:
                for i in self.orders:
                    if i["position"]==position:
                        self.orders.remove(i)
                        order.quantity=res_quantity
                        trade=True     
            elif res_quantity==0 and perc==1:
                for i in self.orders:
                    if i["entry"].quantity ==0:
                        self.orders.remove(i)
                    
                    if i["position"]==position:
                        i["entry"].quantity -= quantity
                        trade=True
                        order.status="OFF"
                        
        elif self.check(order)==False:
            trade=False
            if order.order_type=="Limit Order":
                self.add(order)
            
        return trade
#---------------------------------------------------------------------         
class MOB:
    def __init__(self):
        self.orders=[]

    def add(self,order):
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")
        self.orders.append({"entry":order,"position":datetime.datetime.now().timestamp()})

    def canmatch(self,order,sign=1):
        q=False
        quantity=None
        price=None
        position=None
        mark_quantity = None
        
        
        if len(self.orders)>0:
                if order.side=="Buy":
                    listoforders = [old_order for old_order in sorted(self.orders, key=lambda old_order: \
                                    old_order["position"]) if old_order["entry"].side =="Sell"]
                    sign=-1
                elif order.side=="Sell":
                    listoforders = [old_order for old_order in sorted(self.orders, key=lambda old_order: \
                                    old_order["position"]) if old_order["entry"].side =="Buy"]

                for old_order in listoforders:
                        if order.quantity <= old_order["entry"].quantity:
                            mark_quantity= tuple([min(order.quantity,old_order["entry"].quantity),old_order["position"]])
                            q=True
                        elif order.quantity > old_order["entry"].quantity:
                            mark_quantity= tuple([min(order.quantity,old_order["entry"].quantity),old_order["position"]])
                            q='Partial'
                            
                        if order.order_type=="Limit Order":
                            print("MOB_LO")
                            price=order.price
                        elif order.order_type=="Market Order":
                            print("MOB_MO")
                            price=None
                            
                        if q==True or q=='Partial':
                            break
                        else:
                            continue
                            
                quantity,position=mark_quantity
            
        elif len(self.orders)==0:
            q,quantity,position=(False,None,None)
            
        return q,price,quantity,position
    
    def adjust_book(self,order):
        trade=False
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")
            
        if self.canmatch(order)[0]==True or self.canmatch(order)[0]=='Partial':
            quantity,position = self.canmatch(order)[2:]
            res_quantity=order.quantity-quantity
            perc=order.quantity/quantity
            
            if res_quantity==0 and perc==1:
                for i in self.orders:
                    if i["entry"].quantity ==0:
                        self.orders.remove(i)
                    
                    if i["position"]==position:
                        i["entry"].quantity -= quantity
                        trade=True
                        order.status="OFF"
            elif res_quantity>0 and perc>1:
                for i in self.orders:
                    if i["position"]==position:
                        self.orders.remove(i)
                        order.quantity=res_quantity
                        trade=True
        elif self.canmatch(order)[0]==False:
            trade=False
            if order.order_type=="Market Order":
                self.add(order)
                
        return trade
#--------------------------------------------------------------------------------
class TRADE:
    def __init__(self):
        self.trades=[]
        
    def add(self,book, order):
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")
        if book is (not isinstance(LOB, MOB)): 
            raise ValueError("It must come from a book")
            
        if (isinstance(book, LOB) and book.check(order)==True) \
        or (isinstance(book, LOB) and book.check(order)=='Partial'):
            price = book.match(order)[0]
            quantity = book.match(order)[1]
        elif (isinstance(book, MOB) and book.canmatch(order)[0]==True) \
        or (isinstance(book, MOB) and book.canmatch(order)[0]=='Partial'):
            quantity=book.canmatch(order)[2]
            price=book.canmatch(order)[1]
        if book.adjust_book(order) == True:
            self.trades.append({"position":datetime.datetime.now().timestamp(),"price":price ,"quantity":quantity})
            print({"position":datetime.datetime.now().timestamp(),"price":price ,"quantity":quantity})
            return True
        else:
            return False

#--------------------------------------------------------------------------------

class Engine:
    def __init__(self):
        self.tradebook=TRADE()  
        self.Market_Book=MOB()
        self.Limit_Book=LOB()
            
            
    def process(self, order):
        if not isinstance(order, Order): 
            raise ValueError("It must be an order")

        if order.order_type=="Limit Order":
            while order.status=='ON' and self.Market_Book.adjust_book(order)==True:
                self.tradebook.add(book=self.Market_Book,order=order)
            
            if self.Market_Book.adjust_book(order)==False:
                while order.status=='ON' and self.Limit_Book.adjust_book(order)==True:
                    self.tradebook.add(book=self.Limit_Book,order=order)
        elif order.order_type=="Market Order":
            while order.status=='ON' and self.Limit_Book.adjust_book(order)==True:
                self.tradebook.add(book=self.Limit_Book,order=order)
            
            if self.Limit_Book.adjust_book(order)==False:
                while order.status=='ON' and self.Market_Book.adjust_book(order)==True:
                    self.tradebook.add(book=self.Market_Book,order=order)


In [6]:
a = Order(quantity = 15,side="Buy",price=25)
b = Order(quantity = 10,side="Sell")
c= Order(quantity = 10,side="Buy",price=15)
d= Order(quantity = 10,side="Buy",price=15)
e=Order(quantity = 25,side="Sell",price=10)
Trades=[a,b,c,d,e]

15
10
10
10
25


In [7]:
engine=Engine()

In [8]:
for t in Trades:
    print("---->{}".format(Trades.index(t)))
    engine.process(t)
    print("Limit BOOK")
    for x in engine.Limit_Book.orders:
        print(x["entry"].quantity, x["entry"].side , x["entry"].price)
    print("Market BOOK")   
    for x in engine.Market_Book.orders:
        print(x["entry"].quantity, x["entry"].side, x["entry"].price)

---->0
Limit BOOK
15 Buy 25
Market BOOK
---->1
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
LOB_MO
{'position': 1559127396.571808, 'price': 25, 'quantity': 5}
Limit BOOK
Market BOOK
---->2
Limit BOOK
10 Buy 15
Market BOOK
---->3
LOB_LO
LOB_LO
LOB_LO
Limit BOOK
10 Buy 15
10 Buy 15
Market BOOK
---->4
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
LOB_LO
{'position': 1559127396.574805, 'price': 10, 'quantity': 10}
Limit BOOK
5 Sell 10
Market BOOK
